In [1]:
import tensorflow as tf
import numpy as np
import util
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam, RMSprop

In [2]:
X, Y = util.get_mnist()
N, D = X.shape

Reading in and transforming data...


In [9]:
#input layer is a 2d vector of size N X D
M = 300
i = Input(D, )

#input -> hidden
x = Dense(M, activation = 'relu') (i)

#hidden -> output
x = Dense(D, activation = 'sigmoid') (x)

model = Model(i, x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_2 (Dense)             (None, 300)               235500    
                                                                 
 dense_3 (Dense)             (None, 784)               235984    
                                                                 
Total params: 471,484
Trainable params: 471,484
Non-trainable params: 0
_________________________________________________________________


In [20]:
class Autoencoder(Model) :
    def __init__ (self, model, **kwargs):
        super().__init__(**kwargs)
        self.model = model
        
    def compile(self, opt, loss, **kwargs):
        super().compile(**kwargs)
        self.opt = opt
        self.loss = loss
    
    def train_step(self, batch, **kwargs):
        ##it takes one batch of data and trains on that batch
        
        X, Y = batch
        with tf.GradientTape() as tape :
            X_hat = self.model(X, training = True)
            print(X_hat)
            total_loss = self.loss(Y, X_hat)
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        self.opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {'loss': total_loss}
    
    def test_step(self, batch, **kwargs) :
        X, y = batch
        X_hat = self.model(X, training = False)

        total_loss = self.loss(Y, X_hat)
        return {'loss': total_loss}
    
    def call(self, X, **kwargs) :
        return self.model(X, **kwargs)

In [ ]:
autoencoder = Autoencoder(model)
autoencoder.compile(opt=RMSprop(learning_rate=0.001), loss = tf.keras.losses.BinaryCrossentropy())
autoencoder.fit(X, X, epochs = 1)

In [ ]:
done = False
while not done :
    i = np.random.choice(len(X))
    x = X[i]
    im = model.predict(x.reshape(1, -1)).reshape(28, 28)
    plt.subplot(1, 2, 1)
    plt.imshow(x.reshape(28, 28), cmap = 'gray')
    plt.title('Original')
    plt.subplot(1, 2, 2)
    plt.imshow(im, cmap = 'gray')
    plt.title('Recostruction')
    plt.show()
    
    ans = input('Generate another?')
    if ans and ans[0] in ('n' or 'N') :
        done = True